In [1]:
import pandas as pd
import csv
import re
import datetime
import numpy as np
from urllib.error import HTTPError
import time

In [2]:
#! pip install nytimesarticle
from nytimesarticle import articleAPI
api = articleAPI('wGIMUrqOM3TqL8v0mJT74YAbLjO6p8wl')

In [3]:
fang = ['facebook', 'amazon', 'netflix', 'google']

In [4]:
# helper function to iterate through dates
def daterange(start_date, end_date):
    if start_date <= end_date:
        for n in range( (end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )

In [5]:
def parse_articles(articles):
    news = []
    try:
        for i in articles['response']['docs']:
            dic = {}
            dic['article_id'] = i['_id']
            dic['date'] = i['pub_date'][0:100] # cutting time of day.
            dic['headline'] = i['headline']['main'].encode("utf8")
            dic['url'] = i['web_url']
            dic['lead_paragraph'] = i['lead_paragraph']  
            news.append(dic)
            
    except:
        pass
    return(news)

In [6]:
def return_articles(query,bdate,edate):
    all_articles = []
    for i in range(0,101):
        articles=api.search(q=query,begin_date=bdate,end_date=edate,page=i)
        p_articles=parse_articles(articles)
        all_articles=all_articles+p_articles
    return all_articles

In [ ]:
return_articles('facebook','20191101','20191110')

In [94]:
def return_df(company, start, end):
    name =['article_id','date','headline','url','lead_paragraph']
    res = pd.DataFrame(columns = name)
    for date in daterange(start, end):
        nextdate = (date + datetime.timedelta(1)).strftime("%Y%m%d")
        print(nextdate)
        date = date.strftime("%Y%m%d")
        print(date)
        dailyres = pd.DataFrame(columns = name,data = return_articles(company, date, nextdate))
        print(dailyres)
        res.append(dailyres)
    return res

In [ ]:
start = datetime.date(year = 2019, month = 11, day = 1)
end = datetime.date(year = 2019, month = 11, day = 10)
fb = return_df(fang[0], start, end)
fb

In [8]:
#move the row where lead_paragraph is not availabe
#fb = fb[fb['lead_paragraph']  != "This article is part of David Leonhardt’s newsletter. You can sign up here to receive it each weekday."]
#sort the data by their date in descending order
#sort the data by their date in descending order
pattern = r'([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))'
count = 0
for i in fb['date']:
    i = re.search(pattern, i).group(1)
    i = datetime.datetime.strptime(i, '%Y-%m-%d')
    fb['date'][count] = i
    count += 1
fb.sort_values(by = 'date', ascending = True)

,article_id,date,headline,url,lead_paragraph
92,nyt://article/2b0152cf-932b-5267-8a95-f656cce4...,2018-11-06 00:00:00,b'Facebook Admits It Was Used to Incite Violen...,https://www.nytimes.com/2018/11/06/technology/...,Facebook has long promoted itself as a tool fo...
46,nyt://article/5e9c7c53-fa75-59a8-8d44-8e611740...,2018-11-13 00:00:00,b'Facebook Failed to Police How Its Partners H...,https://www.nytimes.com/2018/11/12/technology/...,Facebook failed to closely monitor device make...
68,nyt://article/b5c7dd41-56b4-523c-9992-042cbfa9...,2018-11-15 00:00:00,b'Facebook and the Fires',https://www.nytimes.com/2018/11/15/opinion/cal...,"When I flew into San Francisco last Sunday, th..."
24,nyt://article/940fa4e0-e597-53d9-b3d1-e426d046...,2018-11-16 00:00:00,b'Democrats Should Un-Friend Facebook',https://www.nytimes.com/2018/11/16/opinion/fac...,In their recent book “LikeWar: The Weaponizati...
29,nyt://article/41c02d43-addb-589c-a49d-1db30724...,2018-11-16 00:00:00,b'How Facebook Has Handled Recent Scandals',https://www.nytimes.com/2018/11/16/opinion/let...,To the Editor:
6,nyt://article/5f8f1936-6980-57dd-8413-83cbf340...,2018-11-16 00:00:00,b'\xe2\x80\x98Facebook Cannot Be Trusted to Re...,https://www.nytimes.com/2018/11/15/opinion/fac...,“Facebook cannot be trusted to regulate itself...
13,nyt://article/71ad4521-ffab-589f-89af-f7434281...,2018-11-17 00:00:00,b'How Plato Foresaw Facebook\xe2\x80\x99s Folly',https://www.nytimes.com/2018/11/16/opinion/fac...,In ancient Egypt there lived a wise king named...
97,nyt://article/999713eb-8f39-5e00-84ed-d9ff6a20...,2018-11-17 00:00:00,b'How to Make Facebook\xe2\x80\x99s \xe2\x80\x...,https://www.nytimes.com/2018/11/17/opinion/fac...,"Facebook’s chief executive, Mark Zuckerberg, h..."
59,nyt://article/0670b72e-8ceb-54b8-8861-bf628ca2...,2018-11-22 00:00:00,"b'On Thanksgiving Eve, Facebook Acknowledges D...",https://www.nytimes.com/2018/11/22/business/on...,Joining a long tradition of companies and camp...
22,nyt://article/7637b4b5-d619-50f2-ad30-e99bdb2d...,2018-11-30 00:00:00,b'The Week in Tech: Facebook\xe2\x80\x99s Thre...,https://www.nytimes.com/2018/11/30/technology/...,"Each week, technology reporters and columnists..."


In [14]:
fb.to_csv(r'fb article.csv', index = None, header = True)

In [9]:
def get_nrc_data():
    nrc = "/Users/wangzhidan/Desktop/Data_Analytics/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count = 0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict


def emotion_analyzer(text,emotion_dict=get_nrc_data()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return [emotion_count.values()]

In [10]:
def return_emotions(stock_df):
    count = 0
    columns = ['anticipation', 'sadness', 'joy', 'negative', 'trust', 'positive', 'surprise', 'disgust', 'anger', 'fear']
    emotions = pd.concat([pd.DataFrame(emotion_analyzer(i), columns=columns) for i in stock_df['lead_paragraph']],
           ignore_index=True)
    
    return emotions

In [ ]:
def return_clean_df(stock_df):
    pattern = r'([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))'
    count = 0
    for i in stock_df['date']:
        i = re.search(pattern, i).group(1)
        i = datetime.datetime.strptime(i, '%Y-%m-%d')
        stock_df['date'][count] = i
        count += 1
    stock_df.sort_values(by = 'date', ascending = True)
    return stock_df

## Facebook article and emotions retrieval

In [11]:
emotions_df = return_emotions(fb)
emotions_df

,anticipation,sadness,joy,negative,trust,positive,surprise,disgust,anger,fear
0,0.017544,0.000000,0.000000,0.017544,0.017544,0.087719,0.035088,0.017544,0.035088,0.017544
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.029412,0.000000,0.029412,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000
4,0.025641,0.051282,0.025641,0.000000,0.000000,0.025641,0.000000,0.000000,0.025641,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000


In [13]:
emotions_df.to_csv(r'fb_emotions.csv', index = None, header = True)

## amazon article and emotions retrieval


In [19]:
am = return_df(fang[1])

In [21]:
am = return_clean_df(am)
am.to_csv(r'am_article.csv', index = None, header = True)

In [22]:
emotions_am = return_emotions(am)

In [23]:
emotions_am.to_csv(r'am_emotions.csv', index = None, header = True)

## netflix article and emotions retrieval

In [26]:
nf = return_df(fang[2])

In [30]:
nf = return_clean_df(nf)
nf.to_csv(r'nf_article.csv', index = None, header = True)

In [31]:
emotions_nf = return_emotions(nf)
emotions_nf.to_csv(r'nf_emotions.csv', index = None, header = True)

## Google article and emotions retrieval

In [33]:
gg = return_df(fang[3])

In [34]:
gg = return_clean_df(gg)
gg.to_csv(r'gg_article.csv', path = , index = None, header = True)

In [40]:
gg

,article_id,date,headline,url,lead_paragraph
0,nyt://article/b2d0b574-72e1-582c-a937-747275c9...,2019-05-30 00:00:00,b'Google Should Google the Definition of \xe2\...,https://www.nytimes.com/2019/05/29/opinion/goo...,"For many companies, hiring full-time employees..."
1,nyt://article/f49f065c-c666-53ec-8dde-704d5d16...,2019-02-21 00:00:00,b'Google Ends Forced Arbitration for All Emplo...,https://www.nytimes.com/2019/02/21/technology/...,SAN FRANCISCO — Google said it would no longer...
2,nyt://article/3f2b5e69-b37d-5a08-9c41-6c778271...,2019-05-21 00:00:00,b'Google Changes Abortion Ad Policy',https://www.nytimes.com/2019/05/21/business/me...,Advertisers planning to run ads on Google refe...
3,nyt://article/3dda4bfd-3544-5436-9376-03a5b4f6...,2019-10-31 00:00:00,b'Google Wants Safeguards for Information in A...,https://www.nytimes.com/2019/10/31/business/go...,SEATTLE — Google fired its opening salvo in wh...
4,nyt://article/ccf70091-5c6c-5266-83cf-5ae3de2a...,2018-12-11 00:00:00,"b'Sundar Pichai, Google\xe2\x80\x99s C.E.O., T...",https://www.nytimes.com/2018/12/11/business/su...,"• Sundar Pichai, Google’s chief executive, tes..."
5,nyt://article/3f229d50-c654-5c36-ab34-b055ff07...,2019-06-06 00:00:00,b'Google to Buy Data Analytics Company Despite...,https://www.nytimes.com/2019/06/06/technology/...,"OAKLAND, Calif. — Just as government officials..."
6,nyt://article/a9038d3b-1e38-5984-9dac-18270b4a...,2019-04-13 00:00:00,b'Google\xe2\x80\x99s Sensorvault Is a Boon fo...,https://www.nytimes.com/2019/04/13/technology/...,Law enforcement officials across the country h...
7,nyt://article/2e5a5213-d054-57b3-b586-a906cebd...,2019-11-15 00:00:00,b'Supreme Court to Hear Google and Oracle Copy...,https://www.nytimes.com/2019/11/15/us/supreme-...,WASHINGTON — The Supreme Court on Friday agree...
8,nyt://article/d3aa8a37-e69f-5c90-bba2-ef0749f7...,2018-11-01 00:00:00,b'Google Walkout: Employees Stage Protest Over...,https://www.nytimes.com/2018/11/01/technology/...,"In New York, more than 3,000 gathered in a cit..."
9,nyt://article/06f4ba41-7d23-5ea2-8c31-519a9a27...,2018-11-02 00:00:00,b'Senior Google Lobbyist Is Stepping Down From...,https://www.nytimes.com/2018/11/02/technology/...,"WASHINGTON — Susan Molinari, who leads Google’..."


In [35]:
emotions_gg = return_emotions(gg)
emotions_gg.to_csv(r'gg_emotions.csv', index = None, header = True)